***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/happydata.csv')
df = df.dropna()

X = df.loc[:, df.columns != 'happy']
y = df['happy']

rus = RandomOverSampler(random_state=42)
X, y = rus.fit_resample(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

params = {
    'C': [1, 5, 10, 25, 50, 100]
}

model = LinearSVC(random_state=42)
cv = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(estimator=model, param_grid=params, n_jobs=-1, cv=cv, scoring='f1_macro')

grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)
rf_best = grid_search.best_estimator_
print("Best Estimator:", rf_best)

Best Parameters: {'C': 1}
Best Score: 0.6077039627039628
Best Estimator: LinearSVC(C=1, random_state=42)


In [ ]:
best = LinearSVC(C=1, random_state=42)
best.fit(X, y)

cv = StratifiedKFold(n_splits=10)

cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
train_predictions = best.predict(X)

train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.68627451 0.25       0.43529412 0.81176471 0.53333333 0.64114833
 0.58333333 0.7        0.35406699 0.46428571]
Mean CV Accuracy: 0.54595010252905
Training Set Accuracy: 0.6038961038961039


##**Extracting weights and intercept values**

In [ ]:
for (intercept, coef) in zip(best.intercept_, best.coef_):
    s = "y = {0:.3f}".format(intercept)
    for (i, c) in enumerate(coef):
      s += " + {0} * x{1}".format(c, i)

    print(s)

y = -1.853 + 0.2758180125288122 * x0 + -0.020837778247036726 * x1 + 0.04610159825340762 * x2 + 0.05467838496210223 * x3 + 0.03307345160305697 * x4 + 0.0590622528185723 * x5


In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['happy'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Happiness_Predicted': predicted_label})
index_class = results_df.index[results_df['Happiness_Predicted'] == 0]

og_x_reset = og_x.reset_index(drop=True)
filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

67


##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

##**Optimizer -- step 1**

In [ ]:
from z3 import *
import csv
import time

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 real variable for X'
    X = [Int(f'x_{i}') for i in range(n)]

    opt.add(And(X[0] >= 3, X[0] <= 4))

    opt.add(And(X[1] >= 3, X[1] <= 4))

    opt.add(And(X[2] >= 3, X[2] <= 5))

    opt.add(And(X[3] >= 2, X[3] <= 5))

    opt.add(And(X[4] >= 2, X[4] <= 5))

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    # Create the inequality constraint W * X + B >= 0
    inequality = dot_product + B > 0
    opt.add(inequality)

    # Calculate the L1 distance between og_x and X
    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]] for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1.csv', names = column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
  x = optimized['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

##**Optimizer -- step 2**

In [ ]:
def step2(W, B, row):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    # Create Z3 int variable for X'
    X = [Int(f'x_{i}') for i in range(n)]

    slack_info = Int('slack_info')
    slack_info_constr = (slack_info >= 0)
    opt.add(And(X[0] > 0, X[0] < 6, X[0] >= 3 - slack_info, X[0] <= 4 + slack_info))
    opt.add(slack_info_constr)

    slack_house_cost = Int('slack_house_cost')
    slack_house_cost_constr = (slack_house_cost >= 0)
    opt.add(And(X[1] > 0, X[1] < 6, X[1] >= 3 - slack_house_cost, X[1] <= 4 + slack_house_cost))
    opt.add(slack_house_cost_constr)

    opt.add(And(X[2] >= 3, X[2] <= 5))

    opt.add(And(X[3] >= 2, X[3] <= 5))

    opt.add(And(X[4] >= 2, X[4] <= 5))

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    objective =  (slack_info + slack_house_cost)
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]] for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Optimized distance: {l1_distance_1}")

        # Calculate the optimized cost
        cost_optimized = model.eval(objective)
        print(f"Optimized cost: {cost_optimized}")

        slack_info = model[slack_info]
        print(f"slack_info value: {slack_info}")

        slack_house_cost = model[slack_house_cost]
        print(f"slack_house_cost value: {slack_house_cost}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_info, slack_house_cost

    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '']

W = w
B = b[0]
output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, cost, cost_optimized, slack_info, slack_house_cost = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_info] + [slack_house_cost]
        writer.writerow(data_row)

Optimized x': [5, 3, 3, 4, 2, 2]
Optimized distance: 1
Optimized cost: 1
slack_info value: 1
slack_house_cost value: 0
Optimized x': [5, 4, 5, 4, 2, 1]
Optimized distance: 1
Optimized cost: 1
slack_info value: 1
slack_house_cost value: 0
Optimized x': [5, 4, 4, 5, 2, 1]
Optimized distance: 1
Optimized cost: 1
slack_info value: 1
slack_house_cost value: 0


In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_info', 'Slack_house_cost']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

[1]
[1]
[1]


##**Optimizer -- step 3**

In [ ]:
def step3(W, B, row, slack_value1, slack_value2):
    n = W.shape[0]

    # Create a Z3 optimization instance
    opt = Optimize()

    X = [Int(f'x_{i}') for i in range(n)]

    opt.add(And(X[0] > 0, X[0] < 6, X[0] >= 3 - slack_value1, X[0] <= 4 + slack_value1))

    opt.add(And(X[1] > 0, X[1] < 6, X[1] >= 3 - slack_value2, X[1] <= 4 + slack_value2))

    opt.add(And(X[2] >= 3, X[2] <= 5))

    opt.add(And(X[3] >= 2, X[3] <= 5))

    opt.add(And(X[4] >= 2, X[4] <= 5))

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    # Calculate the dot product W * X
    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    # Check if there's a solution
    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        # Get the optimized feature vector x_prime
        x_prime_optimized = [model[X[i]] for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective)
        print(f"Distance: {l1_distance_1}")

        # Calculate the optimized distance
        cost_optimized = model.eval(objective)
        print(f"Objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_info'].iloc[i]
    slack_value2 = last['Slack_house_cost'].iloc[i]

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

Optimized x': [5, 3, 3, 2, 2, 5]
Distance: 4
Objective: 4
Optimized x': [5, 3, 3, 3, 3, 5]
Distance: 4
Objective: 4
Optimized x': [5, 4, 3, 3, 5, 5]
Distance: 4
Objective: 4


In [ ]:
# Read the CSV file into a DataFrame
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/New_implementation/Happiness/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

# Apply the function to each row in the DataFrame
df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

[1]
[1]
[1]


## **Average runtimes for l0 & l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

In [ ]:
d3_prior = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv')
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step1_optimized.csv')

d = pd.concat([d1, d3_initial])
l1_runtime = d.loc[:, 'Runtime'].mean()
print("L1 runtime", l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

4.477611940298507


### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/Happiness/Step2.csv')
number_of_slacks_per_record = 2

count_s1 = 0
count_s2 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_info'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_info'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_house_cost'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_house_cost'].iloc[i] == 0:
        count_s2 = 0

    total_count = count_s1 + count_s2
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)

50.0
